In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

一、数据源读取并检查异常值

In [ ]:
#read all datas
os.chdir(r'../input/competitive-data-science-predict-future-sales')
item_categories = pd.read_csv('item_categories.csv')
items = pd.read_csv('items.csv')#
sales_train = pd.read_csv('sales_train.csv')
sample_submission = pd.read_csv('sample_submission.csv')
shops = pd.read_csv('shops.csv')
test = pd.read_csv('test.csv')
train = sales_train.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(item_categories, on='item_category_id', rsuffix='_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)

In [ ]:
x= train['item_cnt_day']
plt.boxplot(x)
plt.show()

In [ ]:
x= train['item_price']
plt.boxplot(x)
plt.show()

In [ ]:
sns.jointplot(x='item_price',y='item_cnt_day',data=train,height=8)
plt.show()

In [ ]:
#1、去除无效多余数据
#剔除异常数据并按照月份聚合
test_shop_ids=test['shop_id'].unique()
test_item_ids = test['item_id'].unique()
lk_train = train[train['shop_id'].isin(test_shop_ids)]
lk_train = lk_train[lk_train['item_id'].isin(test_item_ids)]
lk_train=lk_train[(lk_train['item_cnt_day']<=20)&(lk_train['item_cnt_day']>=0)]
lk_train=lk_train[lk_train.item_price<40000]
train_monthly = lk_train[['date', 'date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'item_cnt_day']]
train_monthly['date']=pd.to_datetime(train_monthly.date)
train_monthly = train_monthly.sort_values('date').groupby(['date_block_num', 'shop_id', 'item_category_id', 'item_id'], as_index=False)
train_monthly = train_monthly.agg({ 'item_price':['mean'],#当月商店的均值
    'item_cnt_day':['sum']})
train_monthly.columns = ['date_block_num', 'shop_id', 'item_category_id', 'item_id','item_price','item_cnt']


In [ ]:
#3、构造补全完整的所有门店34个月里面，月份—商店-商品的组合
shop_ids = train_monthly['shop_id'].unique()
item_ids = train_monthly['item_id'].unique()
empty_df = []
for i in range(34):
    for shop in shop_ids:
        for item in item_ids:
            empty_df.append([i, shop, item])
    
empty_df = pd.DataFrame(empty_df, columns=['date_block_num','shop_id','item_id'])#更改列名
train_monthly = pd.merge(empty_df, train_monthly, on=['date_block_num','shop_id','item_id'], how='left')#合并
train_monthly.fillna(0, inplace=True)#缺失数据填充
train_monthly['year'] = train_monthly['date_block_num'].apply(lambda x: ((x//12) + 2013))#提取年份
train_monthly['month'] = train_monthly['date_block_num'].apply(lambda x: (x % 12))#提取月份


二、EDTA分析销量与各个特征的关联性

In [ ]:
#2、数据EDTA :进行数据探索，分别按照month,category和shop分组，得到各组的销量(求各个维度下销量和均值)画图查看各个指标的相关性情况
#销量月月份间关系
gp_month_mean = train_monthly.groupby(['date_block_num'], as_index=False)['item_cnt'].mean()
gp_month_sum = train_monthly.groupby(['date_block_num'], as_index=False)['item_cnt'].sum()
f, axes = plt.subplots(2, 1, figsize=(15, 8), sharex=True)
sns.lineplot(x="date_block_num", y="item_cnt", data=gp_month_mean, ax=axes[0]).set_title("Monthly mean")
sns.lineplot(x="date_block_num", y="item_cnt", data=gp_month_sum, ax=axes[1]).set_title("Monthly sum")
plt.show()


In [ ]:
#销量与品类间关系
gp_category_mean = train_monthly.groupby(['item_category_id'], as_index=False)['item_cnt'].mean()
gp_category_sum = train_monthly.groupby(['item_category_id'], as_index=False)['item_cnt'].sum()
f, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
sns.barplot(x="item_category_id", y="item_cnt", data=gp_category_mean, ax=axes[0], palette="rocket").set_title("Monthly mean")
sns.barplot(x="item_category_id", y="item_cnt", data=gp_category_sum, ax=axes[1], palette="rocket").set_title("Monthly sum")
plt.show()


In [ ]:
#销量与商店的关系
gp_shop_mean = train_monthly.groupby(['shop_id'], as_index=False)['item_cnt'].mean()
gp_shop_sum = train_monthly.groupby(['shop_id'], as_index=False)['item_cnt'].sum()
f, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
sns.barplot(x="shop_id", y="item_cnt", data=gp_shop_mean, ax=axes[0], palette="rocket").set_title("Monthly mean")
sns.barplot(x="shop_id", y="item_cnt", data=gp_shop_sum, ax=axes[1], palette="rocket").set_title("Monthly sum")
plt.show()

三、特征方程构造

In [ ]:
#4、特征构造
#①构造预测值
train_monthly['item_cnt_month'] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt'].shift(-1) 
#②价格特征（价格与销量的关系使用价格//销量）#最大值、最小值、增量、减量、平均单价、零售价均值
train_monthly['item_price_unit'] = train_monthly['item_price'] // train_monthly['item_cnt']#当月该商店该商品的总销售价//总销量
train_monthly['item_price_unit'].fillna(0, inplace=True)
item_price=lk_train.sort_values('date_block_num').groupby(['item_id'],as_index=False).agg({'item_price':['max','min','mean']})
item_price.columns=['item_id','item_price_max','item_price_min','item_price_mean']
train_monthly=pd.merge(train_monthly,item_price,how='left',on=['item_id'])
train_monthly['price_increase']=train_monthly['item_price']-train_monthly['item_price_min']
train_monthly['price_decrease']=train_monthly['item_price_max']-train_monthly['item_price']

In [ ]:
#③构造销量的时间窗特征
#时间窗可以对数据起到平滑的效果，同时也包含了一定的历史信息。这里我们用时窗构造出min,max,mean以及std特征，并对缺失数据进行零填充
#DataFrame.rolling(window, min_periods=None, center=False, win_type=None, on=None, axis=0, closed=None)设置窗口为3，偏移量1
#③-①时间窗口构造
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
# Max value
f_max = lambda x: x.rolling(window=3, min_periods=1).max()
# Mean value
f_mean = lambda x: x.rolling(window=3, min_periods=1).mean()
# Standard deviation
f_std = lambda x: x.rolling(window=3, min_periods=1).std()
function_name = ['min', 'max', 'mean', 'std']
function_list = [f_min, f_max, f_mean, f_std]
for i in range(len(function_list)):
    train_monthly[('item_cnt_%s' % function_name[i])] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_cnt'].apply(function_list[i])
    train_monthly[('item_cnt_%s' % function_name[i])].fillna(0, inplace=True)
    # Fill the empty std features with 0

##③-②构造滞后历史特征，将历史三个月的数据平移
lag_list = [1, 2, 3]
for lag in lag_list:
    ft_name = ('item_cnt_shifted%s' % lag)
    train_monthly[ft_name] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_cnt'].shift(lag)
    # Fill the empty shifted features with 0
    train_monthly[ft_name].fillna(0, inplace=True)
#③-③构造销量变化特征，通过计算滞后历史特征的变化量来得出：即计算当月往前推3个月的平均每个月的销量变化值（可尝试成环比变化率试一试）
train_monthly['item_trend'] = train_monthly['item_cnt']
for lag in lag_list:
    ft_name = ('item_cnt_shifted%s' % lag)
    train_monthly['item_trend'] =train_monthly['item_trend']- train_monthly[ft_name]
train_monthly['item_trend'] = train_monthly['item_trend']/(len(lag_list) + 1)

四、数据集合划分并处理

In [ ]:
#5、划分测试集、验证集、训练集合
#训练集
train_set = train_monthly[(train_monthly.date_block_num>=3)&(train_monthly.date_block_num<28)]
train_set.dropna(subset=['item_cnt_month'], inplace=True)
train_set.dropna(inplace=True)
#验证集
validation_set = train_monthly[(train_monthly.date_block_num>=28)&(train_monthly.date_block_num<=33)]
validation_set.dropna(subset=['item_cnt_month'], inplace=True)
validation_set.dropna(inplace=True)

In [ ]:
#6、训练集测试集合增加其他常规特征
#商店单款销量均值
gp_shop_mean=train_set.groupby(['shop_id'],as_index=False).agg({'item_cnt_month':'mean'})
gp_shop_mean.columns = ['shop_id','shop_mean']
#品类销量均值
gp_category_mean=train_set.groupby(['item_category_id'],as_index=False).agg({'item_cnt_month':'mean'})
gp_category_mean.columns = ['item_category_id','category_mean']
#商店品类销量均值
gp_shop_category_mean=train_set.groupby(['shop_id','item_category_id'],as_index=False).agg({'item_cnt_month':'mean'})
gp_shop_category_mean.columns = ['shop_id','item_category_id','shop_category_mean']
#商品销量均值
gp_item_mean = train_set.groupby(['item_id'],as_index=False).agg({'item_cnt_month': ['mean']})
gp_item_mean.columns = ['item_id','item_mean']
#商品商店
gp_shop_item_mean = train_set.groupby(['shop_id', 'item_id'],as_index=False).agg({'item_cnt_month': ['mean']})
gp_shop_item_mean.columns = ['shop_id', 'item_id','shop_item_mean']
#每年销量均值
gp_year_mean = train_set.groupby(['year'],as_index=False).agg({'item_cnt_month': ['mean']})
gp_year_mean.columns = ['year','year_mean']
#每月销量均值
gp_month_mean = train_set.groupby(['month'],as_index=False).agg({'item_cnt_month': ['mean']})
gp_month_mean.columns = ['month','month_mean']
#每个月品类销量均值
gp_month_category_mean=train_set.groupby(['month','item_category_id'],as_index=False).agg({'item_cnt_month':'mean'})
gp_month_category_mean.columns = ['month','item_category_id','month_category_mean']


In [ ]:
#将训练集与验证集合与测试集特征合并
train_set = pd.merge(train_set, gp_shop_mean, on=['shop_id'], how='left')
train_set = pd.merge(train_set, gp_category_mean, on=['item_category_id'], how='left')
train_set = pd.merge(train_set, gp_shop_category_mean, on=['shop_id','item_category_id'], how='left')
train_set = pd.merge(train_set, gp_item_mean, on=['item_id'], how='left')
train_set = pd.merge(train_set, gp_shop_item_mean, on=['shop_id', 'item_id'], how='left')
train_set = pd.merge(train_set, gp_year_mean, on=['year'], how='left')
train_set = pd.merge(train_set, gp_month_mean, on=['month'], how='left')
train_set = pd.merge(train_set, gp_month_category_mean, on=['month','item_category_id'], how='left')
#验证及与特征集合并
validation_set = pd.merge(validation_set, gp_shop_mean, on=['shop_id'], how='left')
validation_set = pd.merge(validation_set, gp_category_mean, on=['item_category_id'], how='left')
validation_set = pd.merge(validation_set, gp_shop_category_mean, on=['shop_id','item_category_id'], how='left')
validation_set= pd.merge(validation_set, gp_item_mean, on=['item_id'], how='left')
validation_set = pd.merge(validation_set, gp_shop_item_mean, on=['shop_id', 'item_id'], how='left')
validation_set = pd.merge(validation_set, gp_year_mean, on=['year'], how='left')
validation_set= pd.merge(validation_set, gp_month_mean, on=['month'], how='left')
validation_set= pd.merge(validation_set, gp_month_category_mean, on=['month','item_category_id'], how='left')
#分离出训练集和验证集的输入X和输出Y
#训练集
X_train = train_set.drop(['item_cnt_month', 'date_block_num'], axis=1)#删除月份和月份排序列
Y_train = train_set['item_cnt_month'].astype(int)#获取销量为输出Y值
#测试集
X_validation = validation_set.drop(['item_cnt_month', 'date_block_num'], axis=1)
Y_validation = validation_set['item_cnt_month'].astype(int)
#把商店代码、商品代码、月份、年份转换为数值型
int_features = ['shop_id', 'item_id', 'year', 'month']
X_train[int_features] = X_train[int_features].astype('int32')
X_validation[int_features] = X_validation[int_features].astype('int32')


In [ ]:
#7、测试集处理
#取最后一次记录的商店与商品组合的交易记录特征
latest_records = pd.concat([train_set, validation_set]).drop_duplicates(subset=['shop_id', 'item_id'], keep='last')
latest_records.drop('item_cnt_month', axis=1, inplace=True)
X_test = pd.merge(test, latest_records, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])  
X_test['year'] = 2015
X_test['month'] = 9
X_test[int_features] = X_test[int_features].astype('int32')
X_test = X_test[X_train.columns]

In [ ]:
#8、测试集处理
#取最后一次记录的商店与商品组合的交易记录特征最为测试集特征
latest_records = pd.concat([train_set, validation_set]).drop_duplicates(subset=['shop_id', 'item_id'], keep='last')
latest_records.drop('item_cnt_month', axis=1, inplace=True)
X_test = pd.merge(test, latest_records, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])  
X_test['year'] = 2015
X_test['month'] = 9
X_test[int_features] = X_test[int_features].astype('int32')
X_test = X_test[X_train.columns]

In [ ]:
#9、空值处理
#检查空值并进行填充
X_test.dtypes
X_validation.dtypes
X_train.dtypes
sets = [X_validation, X_test]          
for dataset in sets:
    for shop_id in dataset['shop_id'].unique():
        for column in dataset.columns:
            shop_median = dataset[(dataset['shop_id'] == shop_id)][column].median()
            dataset.loc[(dataset[column].isnull()) & (dataset['shop_id'] == shop_id), column] = shop_median

五、算法预测并计算得分

In [ ]:
#算法一、CatBoost
#此算法优点：不用手动处理类别型特征了、使用了组合类别特征、对称树避免过拟合
import xgboost
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import catboost as cb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import recall_score
# cat_feat表示原始数据的哪几列为类别特征，类别特征需要进行str格式转换
cat_features = ['shop_id','item_id', 'item_category_id', 'year', 'month']
cat_X_train=X_train
cat_X_validation=X_validation
cat_X_test=X_test
cat_X_train[cat_features] = cat_X_train[cat_features].astype(str)
cat_X_validation[cat_features] = cat_X_validation[cat_features].astype(str)
cat_X_test[cat_features] = cat_X_test[cat_features].astype(str)
#参数设置
catboost_model = cb.CatBoostRegressor(
    iterations=500,
    max_ctr_complexity=4,
    random_seed=0,
    od_type='Iter',
    od_wait=25,
    verbose=50,
    depth=4)
#训练模型
catboost_model.fit(
    X_train, Y_train,
    cat_features=cat_features,
    eval_set=(X_validation, Y_validation))
#预测
cat_train_pred = catboost_model.predict(cat_X_train)
cat_val_pred = catboost_model.predict(cat_X_validation)
cat_test_pred = catboost_model.predict(cat_X_test)
#画图查看特征重要性分布
feature_score = pd.DataFrame(list(zip(X_train.columns,catboost_model.feature_importances_)), 
                           columns=['Feature','Score'])
feature_score = feature_score.sort_values(by='Score', ascending=False, inplace=False)#排序
plt.rcParams["figure.figsize"] = (19, 6)
ax = feature_score.plot('Feature', 'Score', kind='bar', color='c')
rects = ax.patches
labels = feature_score['Score'].round(2)
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 0.35, label, ha='center', va='bottom')
plt.show()
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, cat_train_pred))) #测试集均方根误差
print('Validation rmse:', np.sqrt(mean_squared_error(Y_validation, cat_val_pred)))#验证集均方根误差
print('The score of cat prediction is:', catboost_model.score(cat_X_train,Y_train)) # 训练集训练结果得分
print('The recall_score of cat prediction is:', recall_score(Y_train, np.rint(cat_train_pred),average='weighted')) # 训练集回召率

In [ ]:
# 算法二Use only part of features on XGBoost.
import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance
xgb_features = ['item_cnt','item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1', 
                'item_cnt_shifted2', 'item_cnt_shifted3', 'shop_mean', 
                'shop_item_mean', 'item_trend', 'month_mean','month_category_mean','item_mean']

xgb_train = X_train[xgb_features]
xgb_val = X_validation[xgb_features]
xgb_test = X_test[xgb_features]
xgb_model = xgb.XGBRegressor(max_depth=8, 
                         n_estimators=500, 
                         min_child_weight=1000,  
                         colsample_bytree=0.7, 
                         subsample=0.7, 
                         eta=0.3, 
                         seed=0)
xgb_model.fit(xgb_train, 
              Y_train, 
              eval_metric="rmse", #eval_metric=["auc",“rmse”,"logloss"]
              eval_set=[(xgb_train, Y_train), (xgb_val, Y_validation)], 
              verbose=20, 
              early_stopping_rounds=20)

xgb_train_pred =xgb_model.predict(xgb_train)
xgb_val_pred =xgb_model.predict(xgb_val)
xgb_test_pred =xgb_model.predict(xgb_test)

print('Train rmse:', np.sqrt(mean_squared_error(Y_train, xgb_train_pred))) #测试集均方根误差
print('Validation rmse:', np.sqrt(mean_squared_error(Y_validation, xgb_val_pred)))#验证集均方根误差
print('The score of cat prediction is:', xgb_model.score(xgb_train_pred,Y_train)) # 训练集训练结果得分
print('The recall_score of cat prediction is:', recall_score(Y_train, np.rint(xgb_train_pred),average='weighted')) # 训练集回召率
#画图查看特征重要性分布
feature_score = pd.DataFrame(list(zip(X_train.columns,xgb_model.feature_importances_)), 
                           columns=['Feature','Score'])
feature_score = feature_score.sort_values(by='Score', ascending=False, inplace=False)#排序
plt.rcParams["figure.figsize"] = (6, 6)
ax = feature_score.plot('Feature', 'Score', kind='bar', color='c')
ax.set_xlabel('')
rects = ax.patches
labels = feature_score['Score'].round(2)
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width(), height, label, ha='center', va='bottom')
plt.show()

In [ ]:
#算法三随机森林RF
from sklearn.ensemble import RandomForestRegressor
# Use only part of features on random forest.
rf_features = ['shop_id', 'item_id', 'item_cnt',
               'item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1', 
               'shop_mean', 'item_mean', 'item_trend']
rf_train = X_train[rf_features]
rf_val = X_validation[rf_features]
rf_test = X_test[rf_features]
rf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)
rf_model.fit(rf_train, Y_train)
rf_train_pred = rf_model.predict(rf_train)
rf_val_pred = rf_model.predict(rf_val)
rf_test_pred = rf_model.predict(rf_test)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, rf_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_validation, rf_val_pred)))
print('The score of rf prediction is:', rf_model.score(rf_train,Y_train)) # 训练集训练结果得分
print('The recall_score of cat prediction is:', recall_score(Y_train, np.rint(rf_train_pred),average='weighted')) # 训练集回召率

In [ ]:
#算法四：LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
lr_features = ['item_cnt', 'item_cnt_shifted1', 'item_trend', 'item_mean', 'shop_mean']
lr_train = X_train[lr_features]
lr_val = X_validation[lr_features]
lr_test = X_test[lr_features]
lr_scaler = MinMaxScaler()
lr_scaler.fit(lr_train)
lr_train = lr_scaler.transform(lr_train)
lr_val = lr_scaler.transform(lr_val)
lr_test = lr_scaler.transform(lr_test)
lr_model = LinearRegression(n_jobs=-1)
lr_model.fit(lr_train, Y_train)
lr_train_pred=lr_model.predict(lr_train)
lr_val_pred=lr_model.predict(lr_val)
lr_test_pred=lr_model.predict(lr_test)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, lr_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_validation, lr_val_pred)))
print('The score of lr prediction is:', lr_model.score(lr_train,Y_train)) # 训练集训练结果得分
print('The recall_score of cat prediction is:', recall_score(Y_train, np.rint(lr_train_pred),average='weighted')) # 训练集回召率

In [ ]:
#算法五：KNN（无监督聚类）

from sklearn.neighbors import KNeighborsClassifier
knn_features = ['item_cnt', 'item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1',
                'item_cnt_shifted2', 'shop_mean', 'shop_item_mean', 
                'item_trend', 'item_mean']

# Subsample train set (using the whole data was taking too long).
knn_train = X_train[knn_features]
knn_val = X_validation[knn_features]
knn_test = X_test[knn_features]
 
knn_scaler = MinMaxScaler()
knn_scaler.fit(knn_train)
knn_train = knn_scaler.transform(knn_train)
knn_val = knn_scaler.transform(knn_val)
knn_test = knn_scaler.transform(knn_test)
knn_model = KNeighborsClassifier(n_neighbors=9, leaf_size=13, n_jobs=-1)

knn_model.fit(knn_train, Y_train)

knn_train_pred=lr_model.predict(lr_train)
knn_val_pred=lr_model.predict(lr_val)
knn_test_pred=lr_model.predict(lr_test)

print('Train rmse:', np.sqrt(mean_squared_error(Y_train, knn_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_validation, knn_val_pred)))
print('The score of knn prediction is:', knn_model.score(knn_model,Y_train)) # 训练集训练结果得分
print('The recall_score of cat prediction is:', recall_score(Y_train, np.rint(knn_train_pred),average='weighted')) # 训练集回召率
